# Submitted by Arjun Tanpure and Team (email id :- arjuntanpure6@gmail.com)

In [3]:
!pip install opencv-python

In [11]:
import cv2

In [5]:
pip install numpy


Note: you may need to restart the kernel to use updated packages.


In [12]:
import numpy as np

In [7]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [8]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement keras (from versions: none)
ERROR: No matching distribution found for keras


In [13]:
from keras.utils import np_utils

ModuleNotFoundError: No module named 'keras'

In [10]:
pip install os


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [14]:
import os

In [17]:
data_path='C:/Users/KIIT/Desktop/6th sem/Minor Project 6th sem/dataset-20210214T185604Z-001/dataset'


In [18]:
categories = os.listdir(data_path)
labels =[i for i in range(len(categories))]
label_dict =dict(zip(categories,labels)) 
print(label_dict)
print(categories)
print(labels)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/KIIT/Desktop/6th sem/Minor Project 6th sem/dataset-20210214T185604Z-001/dataset'

In [ ]:
img_size=100
data=[]
target=[]

In [ ]:

for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
        resized=cv2.resize(gray,(img_size,img_size))
        data.append(resized)
        target.append(label_dict[category]) 

In [ ]:
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)
new_target=np_utils.to_categorical(target)

In [ ]:
np.save('data',data)
np.save('target',new_target)


In [ ]:
data=np.load('data.npy')
target=np.load('target.npy')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

In [ ]:
model=Sequential()

In [ ]:
model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation='relu'))
model.add(Dense(2,activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.2)

In [ ]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/20
28/28 [==============================] - 14s 482ms/step - loss: 0.7679 - accuracy: 0.4982 - val_loss: 0.6499 - val_accuracy: 0.6773
INFO:tensorflow:Assets written to: model-001.model\assets
Epoch 2/20
28/28 [==============================] - 14s 486ms/step - loss: 0.6218 - accuracy: 0.6385 - val_loss: 0.5451 - val_accuracy: 0.7136
INFO:tensorflow:Assets written to: model-002.model\assets
Epoch 3/20
28/28 [==============================] - 14s 492ms/step - loss: 0.4451 - accuracy: 0.8012 - val_loss: 0.3811 - val_accuracy: 0.8455
INFO:tensorflow:Assets written to: model-003.model\assets
Epoch 4/20
28/28 [==============================] - 14s 497ms/step - loss: 0.3093 - accuracy: 0.8750 - val_loss: 0.3283 - val_accuracy: 0.8591
INFO:tensorflow:Assets written to: model-004.model\assets
Epoch 5/20
28/28 [==============================] - 14s 496ms/step - loss: 0.2129 - accuracy: 0.9170 - val_loss: 0.2528 - val_accuracy: 0.9000
INFO:tensorflow:Assets written to: model-005.model\as

In [ ]:
print(model.evaluate(test_data,test_target))

9/9 [==============================] - 1s 116ms/step - loss: 0.2061 - accuracy: 0.9420
[0.2060931771993637, 0.9420289993286133]


In [ ]:
from keras.models import load_model
import cv2
import numpy as np

In [ ]:

model = load_model('model-015.model')

face_clsfr=cv2.CascadeClassifier('C:/Users/hp/anaconda3/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={1:'MASK',0:'NO MASK'}
color_dict={0:(0,0,255),1:(0,255,0)}

In [ ]:
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()